In [ ]:
#Use pretrained unet network - DONE
#Add information about slice - TODAY
#Are all lungs facing the same way = TOMMOROW?
#Add information about nodule - TOMMOROW
#Lung mask as per simple thresholding? Maybe No. But do measure jaccard/dice for thresholding Vs deep learning
#Add information about patient
#Try 3D Unet
#Process data to have 2 classes - lung, Nodule
#Use Jacckard index instead of dice coefficient
#initialise using tutorial weights
#preprocess to make 0 or 1 in training data
#train on segmented lungs data using unet 
#test on validation set
#Expand data from 160 to zoom on lung
#does every CT scan have patient oriented in same way? TO BE VERIFIED

In [1]:
import SimpleITK as sitk
from tqdm import tqdm
import numpy as np
import csv
import os
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from random import uniform
import bcolz
import time
from keras.layers import merge
from keras.layers.core import Lambda
from keras.models import Model
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Reshape
from keras.layers import Input, Dense
import math
import pandas as pd
from keras.utils import np_utils as u

numGPUs = 1

%matplotlib inline

/home/ubuntu/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%cd '/luna/home/ubuntu/data/masks'
lung_mask_files=glob("*.mhd")
%cd '/dsb/home/ubuntu/data/lungs'
lung_files=glob("*.mhd")
%cd '/luna/home/ubuntu/data/lungs'
%cd ..

/luna/home/ubuntu/data/masks
/dsb/home/ubuntu/data/lungs
/luna/home/ubuntu/data/lungs
/luna/home/ubuntu/data


In [ ]:
def load_itk_image(filename):
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
    return numpyImage, numpyOrigin, numpySpacing

In [3]:
%cd '/dsb/home/ubuntu/data/lungs'
lungs = glob('*.mhd')
num_lungs = len(lungs)
%cd '/luna/home/ubuntu/data/lungs'
%cd '/dsb/home/ubuntu/data/lungs-valid'
lungs_valid = glob('*.mhd')
num_lungs_valid = len(lungs_valid)
%cd '/luna/home/ubuntu/data/lungs'
%cd ..

/dsb/home/ubuntu/data/lungs
/luna/home/ubuntu/data/lungs
/dsb/home/ubuntu/data/lungs-valid
/luna/home/ubuntu/data/lungs
/luna/home/ubuntu/data


In [ ]:
#Load data

## load test lung
img_path = "/dsb/home/ubuntu/data/lungs/1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860.mhd"
numpyImage, numpyOrigin, numpySpacing = load_itk_image(img_path)
print numpyImage.shape
print numpyOrigin
print numpySpacing

In [ ]:
plt.imshow(numpyImage[100])

In [ ]:
## load test Mask
img_path = "masks/1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860.mhd"
numpyImage, numpyOrigin, numpySpacing = load_itk_image(img_path)
print numpyImage.shape
print numpyOrigin
print numpySpacing

In [ ]:
plt.imshow(numpyImage[100])

In [ ]:
mask = np.ndarray([512,512],dtype=np.int8)
mask[:] = 0
mask = numpyImage>0
plt.imshow(mask[100])

In [4]:
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

In [5]:
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_np(y_true,y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [6]:
smooth_j = 1e-12

def jaccard_coef(y_true, y_pred):
    # __author__ = Vladimir Iglovikov
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])
    jac = (intersection + smooth_j) / (sum_ - intersection + smooth_j)
    return K.mean(jac)

def jaccard_coef_int(y_true, y_pred):
    # __author__ = Vladimir Iglovikov
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])
    jac = (intersection + smooth_j) / (sum_ - intersection + smooth_j)
    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -jaccard_coef(y_true, y_pred)

In [ ]:
#Create categorical slice numbers
rawFiles = glob("/luna/home/ubuntu/data/temp/final_slice_indices*.dat")
max_slice_global = 0
for filename in rawFiles:
    f=load_array(filename)
    length = f.shape[0]
    max_slice = np.max(f[range(length),])
    if max_slice_global < max_slice: max_slice_global = max_slice
#Repeat for validation set
rawFiles = glob("/dsb/home/ubuntu/data/temp/final_slice_indices*.dat")
for filename in rawFiles:
    f=load_array(filename)
    length = f.shape[0]
    max_slice = np.max(f[range(length),])
    if max_slice_global < max_slice: max_slice_global = max_slice
max_slice_global

In [ ]:
rawFiles = glob("/luna/home/ubuntu/data/temp/final_slice_indices*.dat")
for filename in rawFiles:
    f=load_array(filename)
    length = f.shape[0]
    a = u.to_categorical(f[0:length,1],max_slice_global+1)
    save_array(filename.replace("final_slice_indices","final_slice_indices_cat"),a)
rawFiles = glob("/dsb/home/ubuntu/data/temp/final_slice_indices*.dat")
for filename in rawFiles:
    f=load_array(filename)
    length = f.shape[0]
    a = u.to_categorical(f[0:length,1],max_slice_global+1)
    save_array(filename.replace("final_slice_indices","final_slice_indices_cat"),a)

In [ ]:
def get_last_layer(s,sliceIndices):
    
    inputs = Input(shape=s)
    sz_inp = Input(shape=sliceIndices.shape)
    x = Dense(256*512, activation='softmax')(sz_inp)
    
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(inputs)
    conv2 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)   
    conv3 = Convolution2D(1,1, 1, activation='sigmoid')(conv2)
    conv4 = MaxPooling2D((2,1))(conv3)

    conv4f = Flatten()(conv4)
    
    dense4 = merge([conv4f,x], 'concat')
    
    dense5 = Reshape((1,512,512)) (dense4)
    
    model = Model(input=[inputs,sz_inp], output=dense5)    
    return model  

t = np.ndarray([723],dtype=np.int8)
lrg_model = get_last_layer((96,512,512),t)  
lrg_model.summary()
lrg_model.compile(Adam(lr=0.001), loss=dice_coef_loss, metrics=[jaccard_coef_int,dice_coef])

In [7]:
def get_last_layer(s):
    
    inputs = Input(shape=s)
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(inputs)
    conv2 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)   
    conv3 = Convolution2D(1,1, 1, activation='sigmoid')(conv2)        
    model = Model(input=inputs, output=conv3)    
    return model  

lrg_model = get_last_layer((96,512,512))  
lrg_model.summary()
lrg_model.compile(Adam(lr=0.001), loss=dice_coef_loss, metrics=[jaccard_coef_int,dice_coef])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 96, 512, 512)  0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, 512, 512)  27680       input_1[0][0]                    
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 512, 512)  9248        convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_3 (Convolution2D)  (None, 1, 512, 512)   33          convolution2d_2[0][0]            
Total params: 36961
_______________________________________________________________________

Generate batches from training and validation datasets

In [8]:
def generate_training_batches(batch_size,data_size):
    for i in range(data_size):
        featuresFiles = glob('/dsb/home/ubuntu/data/temp/final_images_features_%d.dat'%i)
        featuresFiles = np.random.permutation(featuresFiles)
        for f in featuresFiles:
            features = load_array(f)
            masks = load_array(f.replace("images_features","images_masks"))
            sliceIndices = load_array(f.replace("images_features","slice_indices_cat"))
            numRows = features.shape[0]
            count = 0
            while (count < numRows) and (count+batch_size<numRows):
                x = features[count:(count+batch_size)]
                y = masks[count:(count+batch_size)]
                z = sliceIndices[count:(count+batch_size)]
                count += batch_size
                yield (x,y,z)

def generate_validation_batches(batch_size,data_size):
    for i in range(data_size):
        featuresFiles = glob('/luna/home/ubuntu/data/temp/final_images_features_valid_%d.dat'%i)
        for f in featuresFiles:
            features = load_array(f)
            masks = load_array(f.replace("images_features","images_masks"))
            sliceIndices = load_array(f.replace("images_features","slice_indices_cat"))
            numRows = features.shape[0]
            count = 0
            while (count < numRows) and (count+batch_size<numRows):
                x = features[count:(count+batch_size)]
                y = masks[count:(count+batch_size)]
                z = sliceIndices[count:(count+batch_size)]
                count += batch_size
                yield (x,y,z)

In [9]:
def trainMyNw(nb_epoch):
    best_loss_metric = 0
    print "training network using features from:"
    %cd '/dsb/home/ubuntu/data/temp'
    for e in range(nb_epoch):
        print 'epoch %d:'%e
        start = time.clock()
        f = open('/luna/home/ubuntu/log.txt', 'a')
        for x,y,z in generate_training_batches(4,12):
            #lrg_model.fit([x,z], y, batch_size = 4, nb_epoch=1, verbose=0,shuffle=True)
            lrg_model.fit(x, y, batch_size = 4, nb_epoch=1, verbose=0,shuffle=True)
        for x,y,z in generate_validation_batches(4,12):
            #loss_metrics = lrg_model.evaluate([x,z], y, batch_size = 4, verbose=0)
            loss_metrics = lrg_model.evaluate(x, y, batch_size = 4, verbose=0)
        print lrg_model.metrics_names,loss_metrics, "time :", time.clock() - start
        f.write(str(lrg_model.metrics_names))
        f.write(" : ")
        f.write(str(loss_metrics))
        f.write(" : time :")
        f.write(str(time.clock() - start))
        f.write("\n")        
        f.close() 
        if (loss_metrics[0]<best_loss_metric):
            lrg_model.save_weights('/dsb/home/ubuntu/lung_mask_best.hdf5') 
            best_loss_metric = loss_metrics[0]
    return loss_metrics

Train the network

In [ ]:
lrg_model.optimizer.lr = 1e-2
loss_metrics=trainMyNw(2)
lrg_model.optimizer.lr = 1e-3
loss_metrics=trainMyNw(9)
lrg_model.optimizer.lr = 1e-4
loss_metrics=trainMyNw(10)        
lrg_model.save_weights('/dsb/home/ubuntu/lung_mask.hdf5')

In [ ]:
lrg_model.optimizer.lr = 1e-1
loss_metrics=trainMyNw(2)

training network using features from:
/dsb/home/ubuntu/data/temp
epoch 0:


MemoryError: 